# Results from physics-informed training 
    with 2D triangle wake
- Plot sensors, interpolated, predicted.
- Plot sensor loss, divergence, momentum residue (x and y)
- Plot original data compared with predicted, show relative MSE.
- Use probe, plot frequency at locations.

In [ ]:
import sys
sys.path.append('..')
from pathlib import Path
import numpy as np
import h5py
import jax
import jax.numpy as jnp
import yaml
import matplotlib.pyplot as plt

import flowrec.training_and_states as state_utils
import flowrec.data as data_utils
import flowrec.physics_and_derivatives as derivatives

from mpl_toolkits.axes_grid1 import ImageGrid
from scipy.interpolate import RBFInterpolator
from flowrec.utils import simulation
from flowrec import losses
from flowrec.utils.py_helper import slice_from_tuple
from flowrec.utils.system import set_gpu
set_gpu(0,0.5)

In [ ]:
plt.style.use('../flowrec/utils/ppt.mplstyle')
# plt.style.use('../flowrec/utils/a4.mplstyle')

In [ ]:
# results_dir = Path('../local_results/2dtriangle/sweep_loss_classic_fc2branch/dutiful-sweep-2') #
# results_dir = Path('../local_results/2dtriangle/sweep_loss_3_fc2branch/fine-sweep-3') #
# results_dir = Path('../local_results/2dtriangle/sweep_snr20_classic_fc2branch/vital-sweep-34') #
# results_dir = Path('../local_results/2dtriangle/sweep_snr20_3_fc2branch/wise-sweep-23') #

# results_dir = Path('../local_results/2dtriangle/repeat_noisy/snr20_classic/classic-224') #
results_dir = Path('../test/classic-224') #

if results_dir.exists():
    ! ls $results_dir
else:
    raise ValueError('This directory does not exist.')

# Load configurations and results

In [ ]:
with open(Path(results_dir,'config.yml'),'r') as f:
    cfg = yaml.load(f, Loader=yaml.UnsafeLoader)
cfg.data_config.update({'data_dir':'.'+cfg.data_config.data_dir})

with h5py.File(Path(results_dir,'results.h5'),'r') as hf:
    loss_train = np.array(hf.get("loss_train"))
    loss_val = np.array(hf.get("loss_val"))
    loss_div = np.array(hf.get("loss_div"))
    loss_momentum = np.array(hf.get("loss_momentum"))
    loss_sensors = np.array(hf.get("loss_sensors"))

## Load data

In [ ]:
x_base = 132
triangle_base_coords = [49,80]
(ux,uy,pp) = simulation.read_data_2dtriangle(cfg.data_config.data_dir,x_base)
x = np.stack([ux,uy,pp],axis=0)
# remove parts where uz is not zero
s = slice_from_tuple(cfg.data_config.slice_to_keep)
x = x[s]

# information about the grid
datainfo = data_utils.DataMetadata(
    re = cfg.data_config.re,
    discretisation=[cfg.data_config.dt,cfg.data_config.dx,cfg.data_config.dy],
    axis_index=[0,1,2],
    problem_2d=True
).to_named_tuple()

rng = np.random.default_rng(cfg.data_config.randseed)
if cfg.data_config.snr:
    [x_train,x_val,x_test], _ = data_utils.data_partition(x,1,cfg.data_config.train_test_split,REMOVE_MEAN=cfg.data_config.remove_mean,randseed=cfg.data_config.randseed,SHUFFLE=cfg.data_config.shuffle) # Do not shuffle, do not remove mean for training with physics informed loss
    [ux_train,uy_train,pp_train] = np.squeeze(np.split(x_train,3,axis=0))
    [ux_val,uy_val,pp_val] = np.squeeze(np.split(x_val,3,axis=0))
    [ux_test,uy_test,pp_test] = np.squeeze(np.split(x_test,3,axis=0))
    u_train = np.stack((ux_train,uy_train,pp_train),axis=-1)
    u_val = np.stack((ux_val,uy_val,pp_val),axis=-1)
    u_test = np.stack((ux_test,uy_test,pp_test),axis=-1)

    
    std_data = np.std(x,axis=(1,2,3),ddof=1)
    std_n = data_utils.get_whitenoise_std(cfg.data_config.snr,std_data)
    noise_ux = rng.normal(scale=std_n[0],size=x[0,...].shape)
    noise_uy = rng.normal(scale=std_n[1],size=x[1,...].shape)
    noise_pp = rng.normal(scale=std_n[2],size=x[2,...].shape)
    noise = np.stack([noise_ux,noise_uy,noise_pp],axis=0)
    x = x + noise


[x_train_n,x_val_n,x_test_n], _ = data_utils.data_partition(
    x,
    1,
    cfg.data_config.train_test_split,
    REMOVE_MEAN=cfg.data_config.remove_mean,
    randseed=cfg.data_config.randseed,
    SHUFFLE=cfg.data_config.shuffle
) # Do not shuffle, do not remove mean for training with physics informed loss
[ux_train_n,uy_train_n,pp_train_n] = np.squeeze(np.split(x_train_n,3,axis=0))
[ux_val_n,uy_val_n,pp_val_n] = np.squeeze(np.split(x_val_n,3,axis=0))
[ux_test_n,uy_test_n,pp_test_n] = np.squeeze(np.split(x_test_n,3,axis=0))
u_train_n = np.stack((ux_train_n,uy_train_n,pp_train_n),axis=-1)
u_val_n = np.stack((ux_val_n,uy_val_n,pp_val_n),axis=-1)
u_test_n = np.stack((ux_test_n,uy_test_n,pp_test_n),axis=-1)

pb_train = simulation.take_measurement_base(pp_train_n,ly=triangle_base_coords,centrex=0).reshape((cfg.data_config.train_test_split[0],-1))
pb_val = simulation.take_measurement_base(pp_val_n,ly=triangle_base_coords,centrex=0).reshape((cfg.data_config.train_test_split[1],-1))
pb_test = simulation.take_measurement_base(pp_test_n,ly=triangle_base_coords,centrex=0).reshape((cfg.data_config.train_test_split[2],-1))


In [ ]:
## measure
take_observation, insert_observation = cfg.case.observe(cfg.data_config, example_pred_snapshot=u_train[0,...],example_pin_snapshot=pb_train[0,...])
observed_train, train_minmax = take_observation(u_train_n,init=True)
observed_val, val_minmax = take_observation(u_val_n,init=True)
observed_test, test_minmax = take_observation(u_test_n,init=True)

## Visualise input data
Sensors and interpolated flow field.

In [ ]:
t_plt = 20

In [ ]:
observed_plot = np.empty_like(u_test)
observed_plot.fill(np.nan)
observed_plot = insert_observation(jnp.asarray(observed_plot),jnp.asarray(observed_test))

measurements = []
for i in range(observed_plot.shape[-1]):
    measurements.append(observed_plot[t_plt,...,i][~np.isnan(observed_plot[t_plt,...,i])])

In [ ]:
# Get sensor coordinates
sensors_empty = np.empty_like(u_train[[0],...])
sensors_empty.fill(np.nan)

grid_x,grid_y = np.mgrid[0:ux_train.shape[1], 0:ux_train.shape[2]]

gridx1 = np.repeat(grid_x[None,:,:,None],3,axis=3)
gridy1 = np.repeat(grid_y[None,:,:,None],3,axis=3)

idx_x = take_observation(gridx1)
idx_y = take_observation(gridy1)

idx_x = insert_observation(jnp.asarray(sensors_empty),jnp.asarray(idx_x))[0,...]
sensors_loc_x = []
for i in range(idx_x.shape[-1]):
    sensors_loc_x.append(idx_x[...,i][~np.isnan(idx_x[...,i])])

idx_y = insert_observation(jnp.asarray(sensors_empty),jnp.asarray(idx_y))[0,...]
sensors_loc_y = []
for i in range(idx_y.shape[-1]):
    sensors_loc_y.append(idx_y[...,i][~np.isnan(idx_y[...,i])])


In [ ]:
# plot true and interpolation
fig = plt.figure(figsize=(36,15))
axes = []
axes.append(ImageGrid(fig,311,(1,3),share_all=True,cbar_mode='single'))
axes.append(ImageGrid(fig,312,(1,3),share_all=True,cbar_mode='single'))
axes.append(ImageGrid(fig,313,(1,3),share_all=True,cbar_mode='single'))

for i in range(3):
    sensors_loc = np.stack((sensors_loc_x[i].flatten(),sensors_loc_y[i].flatten()),axis=-1)
    # data_interp = griddata(sensors_loc, measurements[i].flatten(), (grid_x,grid_y), method='cubic')
    rbf = RBFInterpolator(sensors_loc,measurements[i])
    data_interp = rbf(np.stack((grid_x.flatten(),grid_y.flatten()),axis=-1)).reshape(grid_x.shape)
    ax = axes[i]
    ax.axes_all[0].imshow(u_test[t_plt,...,i],'jet')
    ax.axes_all[0].set_title('True data')
    ax.axes_all[0].axis('off')
    
    ax.axes_all[1].imshow(u_test_n[t_plt,...,i],'jet')
    ax.axes_all[1].set_title('Noisy data')
    ax.axes_all[1].axis('off')

    im = ax.axes_all[2].imshow(data_interp,'jet')
    ax.axes_all[2].spy(observed_plot[t_plt,...,i],color='k',marker='x',markersize=5,alpha=0.6)
    ax.axes_all[2].set_title('Interpolated')
    ax.axes_all[2].axis('off')
    ax.cbar_axes[0].colorbar(im)
plt.show()

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(np.linspace(1.5,2.5,pb_train.shape[1]),pb_train[0,:])
plt.title('Input to network')
plt.ylabel('Pressure')
plt.xlabel('y')
plt.xlim([1.5,2.5])
plt.show()

# Loss
Plot different losses:
- Sensors
- Divergence
- Momentum residue
- Total physics loss
- Total loss

Also plot relative mse of the entire predicted flow field and compare with interpolated.

In [ ]:
fig,axes = plt.subplots(1,3,figsize=(15,5))
fig.suptitle('Loss history')
axes[0].plot(loss_div,label='Divergence')
axes[0].plot(loss_momentum,label='Momentum')
axes[0].plot(loss_sensors,label='Sensors')
axes[0].legend()
axes[0].set_yscale('log')
axes[1].plot(loss_div+loss_momentum,label='Physics')
axes[1].plot(loss_sensors,label='Sensors')
axes[1].legend()
axes[1].set_yscale('log')
axes[2].plot(loss_train,label='Total loss in training')
axes[2].plot(loss_div+loss_momentum+loss_sensors,label='Total loss without weighting')
axes[2].legend()
axes[2].set_yscale('log')
plt.show()

## Load model

In [ ]:
state = state_utils.restore_trainingstate(results_dir,'state')
jax.tree_util.tree_map(lambda x: print(x.shape),state.params)

In [ ]:
_, make_model = cfg.case.select_model(datacfg=cfg.data_config, mdlcfg=cfg.model_config, traincfg=cfg.train_config)
mdl = make_model(cfg.model_config)

In [ ]:
pb_train[0,:10]

In [ ]:
## Normalise
if cfg.data_config.normalise:
    print(cfg.data_config.normalise)
    [pb_train, pb_val, pb_test], _ = data_utils.normalise(pb_train, pb_val, pb_test, range=[train_minmax[-1],val_minmax[-1],test_minmax[-1]])

In [ ]:
print(pb_train[0,:10])
print(train_minmax)

In [ ]:
rng = jax.random.PRNGKey(10)

pb_train_batch = np.array_split(pb_train,2,0)
pred_train = []
for inn in pb_train_batch:
    pred_train.append(mdl.apply(state.params,rng,inn,TRAINING=False))
pred_train = np.concatenate(pred_train)
pred_test = mdl.apply(state.params,rng,pb_test,TRAINING=False)
if cfg.data_config.normalise:
    pred_train = data_utils.unnormalise_group(pred_train, train_minmax, axis_data=-1, axis_range=0)
    pred_test = data_utils.unnormalise_group(pred_test, test_minmax, axis_data=-1, axis_range=0)


Calculate physics loss

In [ ]:
l_d_test = derivatives.div_field(pred_test[...,:-1],datainfo)
l_m_test = derivatives.momentum_residual_field(pred_test,datainfo)


l_d_train = derivatives.div_field(pred_train[...,:-1],datainfo)
l_m_train = derivatives.momentum_residual_field(pred_train,datainfo)


l_d_true = derivatives.div_field(u_test[...,:-1],datainfo)
l_m_true = derivatives.momentum_residual_field(u_test,datainfo)


l_d_noisy = derivatives.div_field(u_test_n[...,:-1],datainfo)
l_m_noisy = derivatives.momentum_residual_field(u_test_n,datainfo)

# fig = plt.figure(figsize=(12,6))
# grid = ImageGrid(fig,111,(1,3),cbar_mode='each',axes_pad=0.3,cbar_pad=0.05)
# for ax, cax, plt_data in zip(grid.axes_all, grid.cbar_axes, [l_d_true, l_m_true[0,...], l_m_true[1,...]]):
#     im = ax.imshow(np.mean(plt_data,axis=0),'jet')
#     cax.colorbar(im)
# fig.suptitle('Time-averaged divergence, momentum residue x and y for true data.')
# plt.show()

## Plot comparison between prediction and True

In [ ]:
fig = plt.figure(figsize=(12,15))
set_name = ['$u_x$','$u_y$','$p$']

# predicted
grid1 = ImageGrid(fig,131,(3,1),cbar_mode='each',axes_pad=(0.02,0.3))
mse_plane = np.einsum('t x y i -> x y i', (pred_test-u_test)**2)/cfg.data_config.train_test_split[2]
for i, (ax,cax) in enumerate(zip(grid1[:3],grid1.cbar_axes[:3])):
    im = ax.imshow(pred_test[t_plt,:,:,i],'jet')
    ax.set_title(f'Predicted {set_name[i]}')
    cax.colorbar(im)
    ax.grid()

# true
grid2 = ImageGrid(fig,132,(3,1),cbar_mode='each',axes_pad=(0.02,0.3))
mse_plane = np.einsum('t x y i -> x y i', (pred_test-u_test)**2)/cfg.data_config.train_test_split[2]
for i, (ax,cax) in enumerate(zip(grid2[:3],grid2.cbar_axes[:3])):
    im = ax.imshow(u_test[t_plt,:,:,i],'jet')
    ax.set_title(f'True {set_name[i]}')
    cax.colorbar(im)
    ax.grid()

# loss
grid3 = ImageGrid(fig,133,(3,1),cbar_mode='each',axes_pad=(0.02,0.3))
mse_plane = np.einsum('t x y i -> x y i', (pred_test-u_test)**2)/cfg.data_config.train_test_split[2]
for i, (ax,cax) in enumerate(zip(grid3[:3],grid3.cbar_axes[:3])):
    im = ax.imshow(mse_plane[:,:,i],'jet')
    ax.set_title(f'relative error = {losses.relative_error(pred_test[...,i],u_test[...,i]):.5f}')
    cax.colorbar(im)
    ax.grid()

fig.suptitle(f'Columns showing instantaneous prediction, true data at t={t_plt} and time averaged MSE.')

for axpred, axtrue in zip(grid1.axes_all,grid2.axes_all):
    [impred] = axpred.get_images()
    [imtrue] = axtrue.get_images()
    climpred = impred.get_clim()
    climtrue = imtrue.get_clim()
    impred.set_clim(min(climpred[0],climtrue[0]),max(climpred[1],climtrue[1]))
    imtrue.set_clim(min(climpred[0],climtrue[0]),max(climpred[1],climtrue[1]))

plt.show()

Physics loss comparison

Calculate residual of the interpolated data first

In [ ]:
temp_observed_test = np.empty_like(u_test)
temp_observed_test.fill(np.nan)
temp_observed_test = insert_observation(jnp.asarray(temp_observed_test),jnp.asarray(observed_test)) # observed_test is noisy if

In [ ]:
compare_interp = []

_locs = np.stack((grid_x.flatten(),grid_y.flatten()),axis=-1)

for i in range(3):
    sensors_loc = np.stack((sensors_loc_x[i].flatten(),sensors_loc_y[i].flatten()),axis=-1)
    for j in range(cfg.data_config.train_test_split[2]):
        temp_measurement = temp_observed_test[j,...,i][~np.isnan(temp_observed_test[j,...,i])]
        rbf = RBFInterpolator(sensors_loc,temp_measurement.flatten(),kernel='thin_plate_spline')
        _interp = rbf(_locs).reshape(grid_x.shape)
        compare_interp.append(_interp)
        # compare_interp.append(
            # griddata(sensors_loc, temp_measurement.flatten(), (grid_x,grid_y), method='cubic')
        # )
compare_interp = np.array(compare_interp)
compare_interp = np.stack((compare_interp[:cfg.data_config.train_test_split[2],...],compare_interp[cfg.data_config.train_test_split[2]:2*cfg.data_config.train_test_split[2],...],compare_interp[2*cfg.data_config.train_test_split[2]:3*cfg.data_config.train_test_split[2],...]),axis=-1)
nb_elements = np.count_nonzero(~np.isnan(compare_interp[1,...,0]))
compare_interp.shape

In [ ]:
interp_physics = derivatives.div_field(compare_interp[...,:-1],datainfo=datainfo) + np.sum(derivatives.momentum_residual_field(compare_interp,datainfo=datainfo),axis=0)
# interp_physics = np.nan_to_num(interp_physics)
interp_physics.shape
# this is noisy 

In [ ]:
which_value = 0
t_compare = 20
fig = plt.figure(figsize=(10,10))
fig.suptitle(f'Instantaneous comparison at t={t_compare}')
grid = ImageGrid(fig,(0,0,1,0.95),(3,4),cbar_mode='each',axes_pad=(0.5,0.5),cbar_pad=0.005)
ax = grid.axes_all
cax = grid.cbar_axes

clim_min = []
clim_max = []

#first row
ax[0].set_title('True')
im = ax[0].imshow(u_test[t_compare,...,which_value])
clim = im.get_clim()
clim_min.append(clim[0])
clim_max.append(clim[1])
cax[0].colorbar(im)
ax[1].set_title('Predicted')
im = ax[1].imshow(pred_test[t_compare,...,which_value])
clim = im.get_clim()
clim_min.append(clim[0])
clim_max.append(clim[1])
cax[1].colorbar(im)
ax[2].set_title('Interpolated (cubic)')
im = ax[2].imshow(compare_interp[t_compare,...,which_value])
clim = im.get_clim()
clim_min.append(clim[0])
clim_max.append(clim[1])
cax[2].colorbar(im)
ax[3].set_title('Noisy')
im = ax[3].imshow(u_test_n[t_compare,...,which_value])
clim = im.get_clim()
clim_min.append(clim[0])
clim_max.append(clim[1])
cax[3].colorbar(im)

for axis in ax[:4]:
    [im] = axis.get_images()
    im.set_clim(min(clim_min),max(clim_max))


#second row
ax[4].axis('off')
cax[4].axis('off')
ax[5].set_title(f'inst. MSE = {losses.mse(pred_test[t_compare,...,which_value],u_test[t_compare,...,which_value]):.7f}')
im = ax[5].imshow((pred_test[t_compare,...,which_value]-u_test[t_compare,...,which_value])**2,'jet')
cax[5].colorbar(im)
mse_interp = np.sum(np.nan_to_num(compare_interp[t_compare,...,which_value] - u_test[t_compare,...,which_value])**2)/nb_elements
ax[6].set_title(f'inst. MSE = {mse_interp:.7f}')
im = ax[6].imshow((compare_interp[t_compare,...,which_value]-u_test[t_compare,...,which_value])**2,'jet')
cax[6].colorbar(im)
ax[7].set_title(f'inst. MSE = {losses.mse(u_test_n[t_compare,...,which_value],u_test[t_compare,...,which_value]):.7f}')
im = ax[7].imshow((u_test_n[t_compare,...,which_value]-u_test[t_compare,...,which_value])**2,'jet')
cax[7].colorbar(im)


# third row
ax[8].set_title(f'inst. residue\n= {np.mean((l_d_true+l_m_true[0,...]+l_m_true[1,...])[t_compare,...]):.7f}')
# im = ax[8].imshow(np.abs((l_mx_true)[t_compare,...]),'jet')
im = ax[8].imshow(np.abs((l_d_true+l_m_true[0,...]+l_m_true[1,...])[t_compare,...]),'jet')
cax[8].colorbar(im)
ax[9].set_title(f'inst. residue\n= {np.mean((l_d_test+l_m_test[0,...]+l_m_test[1,...])[t_compare,...]):.7f}')
im = ax[9].imshow(np.abs((l_d_test+l_m_test[0,...]+l_m_test[1,...])[t_compare,...]),'jet')
cax[9].colorbar(im)
ax[10].set_title(f'inst. residue\n= {np.sum(np.nan_to_num(interp_physics[t_compare,:,:])/nb_elements):.7f}')
im = ax[10].imshow(np.abs(interp_physics[t_compare,...]),'jet')
cax[10].colorbar(im)
ax[11].set_title(f'inst. residue\n= {np.mean((l_d_noisy+l_m_noisy[0,...]+l_m_noisy[1,...])[t_compare,...]):.7f}')
im = ax[11].imshow(np.abs((l_d_noisy+l_m_noisy[0,...]+l_m_noisy[1,...])[t_compare,...]),'jet')
cax[11].colorbar(im)


for ax in grid:
    ax.grid()

plt.show()

In [ ]:
losses.mse(l_d_noisy)

In [ ]:
datainfo.axis_index

In [ ]:
def vorticity(u, datainfo):
    step_space = datainfo.discretisation[1:]
    axis_space = datainfo.axis_index[1:]
    u = jnp.moveaxis(u[...],-1,0) # move velocity axis to 0
    v_derivative1 = jax.vmap(derivatives.derivative1,(0,None,None),0)
    def _didj(de_fun,inn):
        didj_T = de_fun(inn,datainfo.dx,datainfo.axx).reshape((-1,)+inn.shape)
        for j in range (1,u.shape[0]):
            didj_T = jnp.concatenate(
                (
                didj_T,
                de_fun(inn,step_space[j],axis_space[j]).reshape((-1,)+inn.shape)
                ),
                axis=0
            )
        return didj_T # for de_fun = v_derivative1 and inn=u -> [j,i,t,x,y,z]
    
    dui_dxj_T = _didj(v_derivative1, u) # [j,i,t,x,y,z]

    vort = dui_dxj_T[0,1,...] - dui_dxj_T[1,0,...]

    if len(axis_space) == 3:
        vort = vort + (dui_dxj_T[1,2] - dui_dxj_T[2,1]) + (dui_dxj_T[2,0]-dui_dxj_T[0,2])

    return vort

In [ ]:
vort_train_n = derivatives.vorticity(u_train_n[...,:-1],datainfo)
vort_train = derivatives.vorticity(u_train[...,:-1],datainfo)
vort_pred = derivatives.vorticity(pred_train[...,:-1],datainfo)

In [ ]:
tstep = 15
fig = plt.figure(figsize=(8,7))
grid = ImageGrid(fig,111,(2,3),share_all=True,cbar_location='right',cbar_mode='single')
vmax = np.max(vort_train[0:3*tstep:tstep,...])
vmin = np.min(vort_train[0:3*tstep:tstep,...])
axes = grid.axes_all
for i in range(3):
    im0 = axes[i].imshow(vort_train_n[i*tstep,...].T,vmax=vmax,vmin=vmin)
    im1 = axes[i+3].imshow(vort_pred[i*tstep,...].T,vmax=vmax,vmin=vmin)
    axes[i+3].set(xlabel=f'$t={tstep*i}$',xticks=[])
    cbar = grid.cbar_axes[i].colorbar(im0,label='Vorticity')
axes[0].set(yticks=[],ylabel='True')
axes[3].set(yticks=[],ylabel='Pred')

plt.show()

In [ ]:
fig,axes = plt.subplots(1,3,figsize=(10,3))
axes[0].set_title('mean of the noisy training data\n')
im0 = axes[0].imshow(jnp.mean(u_train_n,axis=0)[...,0],vmax=1.6,vmin=-0.3)
plt.colorbar(im0)
axes[1].set_title('mean of the prediction, \nand location of the sensors')
im1 = axes[1].imshow(jnp.mean(pred_train,axis=0)[...,0],vmax=1.6,vmin=-0.3)
# axes[1].spy(observed_plot[t_plt,...,0],marker='x',markersize=2,c='k')
plt.colorbar(im1)
axes[2].set_title('mean of interpolation\n')
im2 = axes[2].imshow(jnp.mean(compare_interp,axis=0)[...,0],vmax=1.6,vmin=-0.3)
plt.colorbar(im2)
plt.show()
print('Relative L2 of mean: ', losses.mse(np.mean(pred_test,axis=0),np.mean(u_test,axis=0))/losses.mse(np.mean(u_test,axis=0)))

## Statistics

In [ ]:
# probe location
probe_loc = [210,60]

fig = plt.figure(figsize=(5,7))
ax = fig.add_subplot(111)
ax.spy(observed_plot[t_plt,...,0],marker='x',markersize=2,c='k')
ax.imshow(ux_test[0,:,:],'jet',alpha=0.5)
ax.scatter(probe_loc[1],probe_loc[0],zorder=20,c='r')
ax.grid()
plt.show()

In [ ]:
# Frequencies in true data
probe_ux_true = ux_test[:,probe_loc[0],probe_loc[1]]
probe_uy_true = uy_test[:,probe_loc[0],probe_loc[1]]
probe_p_true = pp_test[:,probe_loc[0],probe_loc[1]]

freq = np.fft.fftfreq(len(probe_ux_true),d=0.0002*1250)
f_ux_true = np.fft.fft(probe_ux_true-np.mean(probe_ux_true))/len(probe_ux_true)
f_uy_true = np.fft.fft(probe_uy_true-np.mean(probe_uy_true))/len(probe_uy_true)
f_p_true = np.fft.fft(probe_p_true-np.mean(probe_p_true))/len(probe_p_true)

In [ ]:
# frequencies
probe_ux_pred = pred_test[:,probe_loc[0],probe_loc[1],0]
probe_uy_pred = pred_test[:,probe_loc[0],probe_loc[1],1]
probe_p_pred = pred_test[:,probe_loc[0],probe_loc[1],2]

# freq = np.fft.fftfreq(len(probe_ux_pred),d=0.0002*1250)
f_ux_pred = np.fft.fft(probe_ux_pred-np.mean(probe_ux_pred))/len(probe_ux_pred)
f_uy_pred = np.fft.fft(probe_uy_pred-np.mean(probe_uy_pred))/len(probe_uy_pred)
f_p_pred = np.fft.fft(probe_p_pred-np.mean(probe_p_pred))/len(probe_p_pred)

# frequencies
probe_ux_interp = compare_interp[:,probe_loc[0],probe_loc[1],0]
probe_uy_interp = compare_interp[:,probe_loc[0],probe_loc[1],1]
probe_p_interp = compare_interp[:,probe_loc[0],probe_loc[1],2]

# freq = np.fft.fftfreq(len(probe_ux_interp),d=0.0002*1250)
f_ux_interp = np.fft.fft(probe_ux_interp-np.mean(probe_ux_interp))/len(probe_ux_interp)
f_uy_interp = np.fft.fft(probe_uy_interp-np.mean(probe_uy_interp))/len(probe_uy_interp)
f_p_interp = np.fft.fft(probe_p_interp-np.mean(probe_p_interp))/len(probe_p_interp)


In [ ]:
# fig,ax = plt.subplots(1,3,figsize=(15,4))
# ax[0].set(title='Frequency spectrum of prediction',xlabel='f',xlim=[0,2])
# ax[0].plot(freq[:int(len(freq)/2)], np.abs(f_ux_pred)[:int(len(freq)/2)],label='ux')
# ax[0].plot(freq[:int(len(freq)/2)], np.abs(f_uy_pred)[:int(len(freq)/2)],label='uy')
# ax[0].plot(freq[:int(len(freq)/2)], np.abs(f_p_pred)[:int(len(freq)/2)],label='p')
# ax[0].legend()
# ax[0].minorticks_on()
# ax[0].grid(True,which='both',axis='x')

# ax[1].set(title='Frequency spectrum of interpolated data',xlabel='f',xlim=[0,2])
# ax[1].plot(freq[:int(len(freq)/2)], np.abs(f_ux_interp)[:int(len(freq)/2)],label='ux')
# ax[1].plot(freq[:int(len(freq)/2)], np.abs(f_uy_interp)[:int(len(freq)/2)],label='uy')
# ax[1].plot(freq[:int(len(freq)/2)], np.abs(f_p_interp)[:int(len(freq)/2)],label='p')
# ax[1].legend()
# ax[1].minorticks_on()
# ax[1].grid(True,which='both',axis='x')

# ax[2].set(title='Frequency spectrum of true data',xlabel='f',xlim=[0,2])
# ax[2].plot(freq[:int(len(freq)/2)], np.abs(f_ux_true)[:int(len(freq)/2)],label='ux')
# ax[2].plot(freq[:int(len(freq)/2)], np.abs(f_uy_true)[:int(len(freq)/2)],label='uy')
# ax[2].plot(freq[:int(len(freq)/2)], np.abs(f_p_true)[:int(len(freq)/2)],label='p')
# ax[2].legend()
# ax[2].minorticks_on()
# ax[2].grid(True,which='both',axis='x')

fig, ax = plt.subplots(1,1,figsize=(10,5))
ax.set(title='Frequency spectrum', xlabel='f', xlim=[0,2])
ax.plot(freq[:int(len(freq)/2)], np.abs(f_ux_pred)[:int(len(freq)/2)],label='ux_pred',c='g',linestyle='-')
ax.plot(freq[:int(len(freq)/2)], np.abs(f_uy_pred)[:int(len(freq)/2)],label='uy_pred',c='k',linestyle='-')
ax.plot(freq[:int(len(freq)/2)], np.abs(f_p_pred)[:int(len(freq)/2)],label='p_pred',c='r',linestyle='-')
ax.plot(freq[:int(len(freq)/2)], np.abs(f_ux_interp)[:int(len(freq)/2)],label='ux_interp',c='g',linestyle='--')
ax.plot(freq[:int(len(freq)/2)], np.abs(f_uy_interp)[:int(len(freq)/2)],label='uy_interp',c='k',linestyle='--')
ax.plot(freq[:int(len(freq)/2)], np.abs(f_p_interp)[:int(len(freq)/2)],label='p_interp',c='r',linestyle='--')
ax.plot(freq[:int(len(freq)/2)], np.abs(f_ux_true)[:int(len(freq)/2)],label='ux',c='g',linestyle='',marker='x')
ax.plot(freq[:int(len(freq)/2)], np.abs(f_uy_true)[:int(len(freq)/2)],label='uy',c='k',linestyle='',marker='x')
ax.plot(freq[:int(len(freq)/2)], np.abs(f_p_true)[:int(len(freq)/2)],label='p',c='r',linestyle='',marker='x')
ax.minorticks_on()
ax.legend()
ax.grid(True,which='both',axis='x')
ax.set_xlim([0,0.75])

plt.show()

# Overall comparison


Compare mean squared error

In [ ]:
print('Relative MSE for prediction in testing: ', losses.relative_error(pred_test,u_test))
print('Relative MSE for prediction in training: ', losses.relative_error(pred_train,u_train))
print('Relative MSE for interpolation in testing: ', losses.relative_error(compare_interp,u_test))
print('Relative MSE for noisy testing set: ', losses.relative_error(u_test_n,u_test))

In [ ]:
## residue averaged over time
rmse_true = np.mean((l_d_true+l_m_true[0,...]+l_m_true[1,...])**2)
rmse_pred = np.mean((l_d_test+l_m_test[0,...]+l_m_test[1,...])**2)
rmse_train = np.mean((l_d_train+l_m_train[0,...]+l_m_train[1,...])**2)
rmse_interp = np.sum(np.nan_to_num(interp_physics**2))/nb_elements
rmse_noisy = np.mean((l_d_noisy+l_m_noisy[0,...]+l_m_noisy[1,...])**2)
print('Residual**2 averaged over space and time:')
print(f'True data: {rmse_true:.7f}')
print(f'predicted training data: {rmse_train:.7f}')
print(f'predicted testing data: {rmse_pred:.7f}')
print(f'interp data: {rmse_interp:.7f}')
print(f'noisy data: {rmse_noisy:.7f}')

In [ ]:
percent_energy = np.sum(pred_test**2)/np.sum(u_test**2)
print('% energy of the testing set: ', percent_energy)

In [ ]:
interp_test_nonan = []
for i in range(3):
    mask = ~np.isnan(compare_interp[0,...,i])
    interp_test_nonan.append(compare_interp[...,i][:,mask])

fig,axes = plt.subplots(1,3,figsize=(8,4))
for i,var in zip(range(3),['u','v','p']):
    counts_true,bins_true = np.histogram(u_test[...,i].flatten()-np.mean(u_test[...,i].flatten()), density=True, bins='auto')
    axes[i].stairs(counts_true,bins_true,label='true',linewidth=3, color='#808080',alpha=0.5)
    counts,bins = np.histogram(pred_test[...,i].flatten()-np.mean(pred_test[...,i].flatten()), density=True, bins='auto')
    axes[i].stairs(counts,bins,label='prediction',color='k')
    counts,bins = np.histogram(interp_test_nonan[i].flatten()-np.mean(interp_test_nonan[i].flatten()), density=True, bins='auto')
    axes[i].stairs(counts,bins,label='interpolation',color='r',linestyle='--')
    axes[i].set(title=var)
axes[0].set_ylabel('Probability density')
axes[2].legend()
plt.show()


freq_data_overall = np.einsum('t x y u -> t u', np.abs(np.fft.fft(u_test-np.mean(u_test,axis=0,keepdims=True),axis=0)))
# freq_data_overall = freq_data_overall / np.std(freq_data_overall,axis=0)
freq_pred_overall = np.einsum('t x y u -> t u', np.abs(np.fft.fft(pred_test-np.mean(pred_test,axis=0,keepdims=True),axis=0)))
# freq_pred_overall = freq_pred_overall / np.std(freq_pred_overall,axis=0)
freq_interp = []
for i in range(3):
    _interp = np.einsum('t n -> t', np.abs(np.fft.fft(interp_test_nonan[i]-np.mean(interp_test_nonan[i]),axis=0)))
    # _interp = _interp / np.std(_interp)
    freq_interp.append(_interp)

fig,axes = plt.subplots(1,3,figsize=(8,4),sharex=True)
for i,var in zip(range(3),['u','v','p']):
    axes[i].plot(freq[:int(len(freq)/2)], freq_data_overall[:int(len(freq)/2),i],label='true',linewidth=3,color='#808080',alpha=0.5)
    axes[i].plot(freq[:int(len(freq)/2)], freq_pred_overall[:int(len(freq)/2),i],label='prediction',color='k',linewidth=1)
    axes[i].plot(freq[:int(len(freq)/2)], freq_interp[i][:int(len(freq)/2)],label='interpolation',color='r',linewidth=1,linestyle='--')
    axes[i].set(title=var,xlabel='frequency',xlim=[0,1.0])
axes[2].legend()
axes[0].set(ylabel='energy')
plt.show()

In [ ]:
fftn_pred = np.fft.fftn(pred_test-np.mean(pred_test,axis=0,keepdims=True),axes=[1,2])
fftn_true = np.fft.fftn(u_test-np.mean(u_test,axis=0,keepdims=True),axes=[1,2])

freqx = np.fft.fftfreq(250,d=datainfo.dx)
freqx = freqx[:int(len(freqx)/2)]
freqy = np.fft.fftfreq(129,d=datainfo.dy)
freqy = freqy[:int(len(freqy)/2)]

In [ ]:
which_value = 2
fig,axes = plt.subplots(1,2,figsize=(8,4),sharex=True,sharey=True)

im0 = axes[0].imshow(np.log(np.mean(np.abs(fftn_pred[:,:,:,which_value][:,:len(freqx),:len(freqy)]),axis=0)), 
    extent=[freqy[0],freqy[-1],freqx[0],freqx[-1]]
)
axes[0].set(xlabel='$f_y$ (Hz)',ylabel='$f_x$ (Hz)',title='Prediction')
plt.colorbar(im0,label='$log((m/s)^2/Hz)$')

im1 = axes[1].imshow(np.log(np.mean(np.abs(fftn_true[:,:,:,which_value][:,:len(freqx),:len(freqy)]),axis=0)), 
    extent=[freqy[0],freqy[-1],freqx[0],freqx[-1]]
)
plt.colorbar(im1,label='$log((m/s)^2/Hz)$')
axes[1].set(title='True')
plt.show()



In [ ]:
"🤯 END 🤯"